In [ ]:
# Install required packages in the Colab runtime
!pip install pdf2image reportlab > /dev/null
!apt install poppler-utils > /dev/null
!pip install daltonlens > /dev/null
print("Successfully installed all required packages!")

In [ ]:
!git clone https://github.com/jgmatt/color-accessibility-checker.git

In [ ]:
import sys
sys.path.insert(0,'/content/color-accessibility-checker')

In [ ]:
from utils import upload_files, settings_widgets, process_and_display

# Upload your figure here

In [ ]:
upload_widget = upload_files()

# Choose settings here

In [ ]:
cb_type_widget, severity_widget, pdf_page_widget = settings_widgets()
display(cb_type_widget, severity_widget, pdf_page_widget)

# Run to process your image

In [ ]:
process_and_display(upload_widget, cb_type_widget, severity_widget, pdf_page_widget)